# Data preparation: Embeddings and Indexing
Este notebook refere-se à etapa de preparação dos dados, contemplando o pipeline de vetorização dos dados textuais e armazenamento dos vetores semânticos para consulta. Os passos aqui apresentados são:
- Chunking dos documentos
- Vetorização (embeddings)
- Teste e indexação com FAISS
- Indexação e persistência em banco Qdrant
- Consulta de checagem da indexação

Para esta etapa, **será utilizado apenas o dataset Quati, com a tabela de `passages`**, tendo em vista que é esse conjunto de dados que contém os documentos necessários para recuperação da informação e é conjunto que define o problema de negócio proposto.

Como a coleta dos dados foi feito diretamente do Hugging Face, **o pré-processamento textual (limpeza e normalização) foi replicado do notebook `02_preprocessing.ipynb`** para padronização e consistência nas transformações aplicadas no projeto.

## Instalações e importações

In [1]:
# Check CUDA version
!nvcc --version

/bin/bash: line 1: nvcc: command not found


In [2]:
# Instalações
!pip install "datasets<4.0.0"
!pip install faiss-cpu #faiss-gpu-cu12
!pip install qdrant-client

In [3]:
# Imports
# Utils
import warnings
from typing import List
from multiprocessing import cpu_count
cpu_c = cpu_count()
print(f"CPU count: {cpu_c}")
warnings.filterwarnings("ignore")

# NLP
import re
import unicodedata

# Manipulação dos dados
import numpy as np
from datasets import load_dataset

# Embeddings
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

# Indexação e persistência vetorial
import faiss
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

CPU count: 2


## Carregamento dos dados e limpeza textual

In [4]:
# Load Quati dataset
quati_ds = load_dataset("unicamp-dl/quati", "quati_1M_passages", trust_remote_code=True)["quati_1M_passages"]

print(quati_ds)
print("-" * 100)
quati_ds[:3]

Dataset({
    features: ['passage_id', 'passage'],
    num_rows: 1000000
})
----------------------------------------------------------------------------------------------------


{'passage_id': ['clueweb22-pt0000-00-00003_1',
  'clueweb22-pt0000-00-00003_2',
  'clueweb22-pt0000-00-00003_3'],
 'passage': ['Se você precisar de ajuda, visite o website nacional sobre a COVID-19 ou ligue para a linha de apoio à COVID-19 808 24 24 24 Perguntas mais frequentes Posso viajar entre Sintra e Cascais? Quais são as restrições de viagem em Cascais? Qual o número de telefone de apoio para a COVID 19 em Cascais? Preciso utilizar máscara facial no transporte público em Cascais? A prática do distanciamento social é compulsória em Cascais? O que eu devo fazer caso apresente sintomas da COVID-19 quando chegar em Cascais? Última atualização: 25 Abr 2022 Aplicam-se exceções, para detalhes completos: European Union. Estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à COVID-19. Esta informação é compilada a partir de fontes oficiais. Ao melhor de nosso conhecimento, está correta de acordo com a última atualização. Visite Avisos de Viag

In [5]:
# Function definition to clean text
HTML_TAG = re.compile(r"<[^>]+>")
URL = re.compile(r"https?://\S+|www\.\S+")
EMAIL = re.compile(r"\S+@\S+")
MULTISPACE = re.compile(r"\s+")

def clean_text(text: str) -> str:
    """
    Realiza limpeza textual baseada em expressões regulares.
    As transformações aplicadas são:
    - Normalização Unicode
    - Remoção de ruídos (como sequência de "===" ou " ---")
    - Remoção de HTML, URLs e E-mails
    - Remoção de caracteres não-principais e de controle
    - Normalização de espaços

    Params:
        text (str): Texto a ser limpo.

    Returns:
        str: Texto limpo.
    """
    if not text: return ""

    # Unicode normalization (NFKC)
    text = unicodedata.normalize('NFKC', text)

    # Remove noise (repeated sequences such as "=====" or "----")
    text = re.sub(r'([^a-zA-Z0-9\s])\1{3,}', ' ', text)

    # Remove HTML, URLs, E-mails
    text = HTML_TAG.sub('', text)
    text = URL.sub('', text)
    text = EMAIL.sub('', text)

    # Clean non-printable characters and control
    text = "".join(ch for ch in text if unicodedata.category(ch)[0] != "C")

    # Spaces normalization (useful for keeping cleaned chunks in RAG)
    text = MULTISPACE.sub(' ', text).strip()

    return text.lower()

In [6]:
# Clean Quati passages
def clean_quati_passages(batch):
    batch["passage"] = [clean_text(p) for p in batch["passage"]]
    return batch

quati_ds = quati_ds.map(
    clean_quati_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_c
)

quati_ds["passage"][:2]

['se você precisar de ajuda, visite o website nacional sobre a covid-19 ou ligue para a linha de apoio à covid-19 808 24 24 24 perguntas mais frequentes posso viajar entre sintra e cascais? quais são as restrições de viagem em cascais? qual o número de telefone de apoio para a covid 19 em cascais? preciso utilizar máscara facial no transporte público em cascais? a prática do distanciamento social é compulsória em cascais? o que eu devo fazer caso apresente sintomas da covid-19 quando chegar em cascais? última atualização: 25 abr 2022 aplicam-se exceções, para detalhes completos: european union. estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à covid-19. esta informação é compilada a partir de fontes oficiais. ao melhor de nosso conhecimento, está correta de acordo com a última atualização. visite avisos de viagem rome2rio para ajuda geral. perguntas & respostas qual a maneira mais econômica de ir de sintra para cascais? qual a maneira

## Chunking
Para realização do chunking dos documentos, é obrigatoriamente necessário **utilizar o mesmo modelo de vetorização dos textos na tokenização** (útil para o chunking de dados por tokens, que mantém a semântica no trecho). No tópico de vetorização, tem o detalhamento do modelo selecionado.

Os chunks foram delimitados com até **256 tokens** com **overlap de 64 tokens** entre os chunks.

Esse modelo é relativamente leve e pouco custoso, além de possuir semântica multilinguística e ser focado para busca semântica.


In [7]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "intfloat/multilingual-e5-small",
    # device="cuda"
)

print(f"Model selected: {tokenizer.name_or_path}")
print(f"Tokenizer max length: {tokenizer.model_max_length}")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")

In [8]:
# Function to chunk text
def chunk_text(text: str, max_tokens: int = 256, overlap: int = 64) -> List[str]:
    """
    Divide um texto em trechos (chunks) a partir de tokens, mantendo um overlap entre os trechos.
    Cada trecho é representado por uma lista de tokens contendo até max_tokens palavras.
    O overlap entre os trechos é definido por overlap tokens.

    Params:
        text (str): Texto a ser dividido em trechos.
        max_tokens (int): Número máximo de tokens por trecho.
        overlap (int): Número de tokens de overlap entre os trechos.

    Returns:
        List[str]: Lista de trechos do texto.
    """
    # Create tokens
    tokens = tokenizer(
        text,
        truncation=False,
        return_offsets_mapping=False,
        add_special_tokens=False,
    )["input_ids"]

    # Set chunks
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        # Set chunk ids
        chunk_ids = tokens[i:i + max_tokens]
        if not chunk_ids:
            continue

        # Get chunks
        chunk = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk)
    return chunks


# Function alternative: uses offsets mapping to split the chunks by tokens
def chunk_by_tokens(text, chunk_size=384, overlap=64):
    tokens = tokenizer(
        text,
        return_offsets_mapping=True,
        add_special_tokens=False,
    )
    input_ids = tokens["input_ids"]
    offsets = tokens["offset_mapping"]

    chunks = []
    start = 0

    while start < len(input_ids):
        end = start + chunk_size
        chunk_offsets = offsets[start:end]
        char_start = chunk_offsets[0][0]
        char_end = chunk_offsets[-1][1]
        chunks.append(text[char_start:char_end])
        start += chunk_size - overlap

    return chunks

In [ ]:
# Applying chunks into documents
def apply_chunks(batch):
    batch["chunks"] = [chunk_text(doc) for doc in batch["passage"]]
    return batch

quati_ds = quati_ds.map(
    apply_chunks,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_c
)

quati_ds.select_columns(["chunks", "passage"])[:2]

Map (num_proc=2):   0%|          | 0/1000000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Flatten the chunks for indexing
texts = []
metadatas = []

for row in quati_ds:
    for idx, chunk in enumerate(row["chunks"]):
        texts.append(chunk)
        metadatas.append({
            "passage_id": row["passage_id"],
            "chunk_id": idx
        })

print(f"Total of chunks: {len(texts)}")

## Vetorização dos dados (embeddings)
Na seleção do modelo para vetorização dos textos, foram buscado modelos recentes no Hugging Face que apresentem boas perfomances e precisão, com semântica multilinguística e, principalmente, com baixa latência e alta velocidade em inferência e no uso.

A partir de uma breve pesquisa de modelos para embedding, o modelo que melhor atende as especificações necessárias foi o modelo `Multilingual-E5-Small`.

Dentre os modelos considerados, mantendo o foco em velocidade e baixo custo, abaixo segue a descrição das possibilidades avaliadas:

| **Modelo**                     | **Descrição**                                                                                                      | **Latência/Velocidade**                                        | **Memória**                              | **Recursos/Observações**                                                                 |
|---------------------------------|---------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------|------------------------------------------|-------------------------------------------------------------------------------------------|
| **Multilingual-E5-Small**       | Modelo "all-rounder" otimizado para buscas semânticas em mais de 100 idiomas.                                       | "Campeão de velocidade", latência de 16ms, até 7x mais rápido que modelos baseados em LLMs. | Menos de 200MB de RAM                   | Ideal para RAGs em produção que exigem alta taxa de transferência (QPS).                 |
| **EmbeddingGemma-300M**        | Lançado em setembro de 2025 pelo Google, baseado na arquitetura Gemma 3.                                             | Sub-segundo, projetado para execução local e on-device.         | Requer apenas 300MB de RAM (cai para 100-200MB com quantização). | Suporta Matryoshka (MRL), permitindo reduzir vetores de 768 para 128 dimensões sem grande perda de precisão. |
| **BGE-Small-v1.5**             | Versão compacta da família BAAI, focada em tarefas de recuperação técnica e RAG.                                     | Latência competitiva abaixo de 30ms.                           | Semelhante ao E5-Small, utiliza dimensões reduzidas (384). | Melhor para quem precisa de estabilidade na distribuição de similaridade (evita scores inflados). |
| **MPNet-Base-v2 (Multilingual)** | Modelo clássico para detecção de paráfrases e similaridade curta.                                                   | Rápido (~25ms), mas superado pelos modelos acima em precisão de busca. | ~420MB                                  | Limitado a contextos curtos (128 tokens), dificultando uso em RAGs com parágrafos longos. |


In [ ]:
# Initialize model
model = SentenceTransformer(
    "intfloat/multilingual-e5-small",
    # device="cuda"
)

In [ ]:
# Create embeddings
embeddings = model.encode(
    texts,
    show_progress_bar=True,
    batch_size=1024,
    normalize_embeddings=True   # For equalizing search methods
)

print(f"Shape of embeddings: {embeddings.shape}")
print(f"Quantidade de vetores: {embeddings.shape[0]}")
print(f"Quantidade de dimensões: {embeddings.shape[1]}")

## Indexação e validação local
A critério de aprendizado, a indexação feita nesse momento utilizará a biblioteca `FAISS` que permite buscas de similaridades rápidas através da crição de estruturas de dados (índices) que organizam os vetores de dados.

Importante destacar que o FAISS é nativamente compatível com bibliotecas como Langchain e LlamaIndex para utilização em pipelines RAG.

O método de indexação utilizado foi o `IndexFlat` que permite armazenamento dos vetores sem compressão e busca 100% exata comparando a consulta com cada vetor.

Além desse método, é importante considerar o método `Inverted File Index` (IVF) que separa o espaço vetorial em clusters com k-means, e método HNSW baseado em grafo multicamadas.

In [ ]:
# Create index
dim = embeddings.shape[1]   # Vector dimension for embeddings
index = faiss.IndexFlatIP(dim)  # Index Flat method uses Brute force search

In [ ]:
# Transfer index to GPU (index 0 for GPU)
# gpu_res = faiss.StandardGpuResources()  # Create GPU resource

# index = faiss.index_cpu_to_gpu(gpu_res, 0, index)

In [ ]:
# Add vectors (embeddings) into index
index.add(embeddings)

print(f"Total of indexed vectors: {index.ntotal}")

In [ ]:
# Quick search (top 5 chunks)
k = 5
distances, indices = index.search(np.array([embeddings[0]]), k)

print(f"Distances: {distances}")
print(f"Indices: {indices}")

In [ ]:
# Persist index and embeddings
faiss.write_index(index, "faiss_index.faiss")   # Index
np.save("embeddings.npy", embeddings)   # Embeddings

## Persistência dos dados
Para persistência real dos embeddings e utilização de um Vector DB dentro da aplicabilidade do projeto, optamos por utilizar o Qdrant.

In [ ]:
# Initialize DB
client = QdrantClient(":memory:")

In [ ]:
# Create colletion for chunks
client.recreate_collection(
    collection_name="quati_chunks",
    vectors_config=VectorParams(
        size=dim,   # Size = Embeddings dimension
        distance=Distance.COSINE    # Method to calculate distance
    )
)

In [ ]:
# Upsert data
points = [
    PointStruct(
        id=idx,
        vector=embeddings[idx].tolist(),
        payload={
            "text": texts[idx],
            "metadata": metadatas[idx]
        }
    )
    for idx in range(len(texts))
]

client.upsert(
    collection_name="quati_chunks",
    points=points
)

## Query check
Abaixo, segue uma checagem rápida da busca vetorial através do Qdrant.

In [ ]:
# Define a query
query = clean_text("O que é o Imposto de Renda para Pessoa Física?")
query_emb = model.encode(query, normalize_embeddings=True)

print(f"Query: {query}")
print(f"Query embedding: {query_emb}")

In [ ]:
# Search for chunks with similarity
hits = client.search(
    collection_name="quati_chunks",
    query_vector=query_emb.tolist(),
    limit=k
)

for h in hits:
    print(f"Score: {h.score}")
    print(f"Text: {h.payload['text']}")
    print(f"Metadata: {h.payload['metadata']}")
    print("-" * 100)